In [2]:
!pip install deap


In [4]:
# Importing required modules
# DEAP (Distributed Evolutionary Algorithms in Python) is a framework for evolutionary computation.
# We're using its base functionality, creation capabilities, and some built-in tools.
from deap import base, creator, tools

# The random module will be used for generating random numbers and choices.
import random

# numpy is a library for numerical computing in Python.
import numpy as np

# The function below converts an integer into its corresponding binary representation.
# This binary string has a fixed length determined by num_bits.
# Example: int_to_bin_str(5, 5) => '00101'
def int_to_bin_str(value, num_bits=25):
    # Convert the integer value to a binary string.
    # The bin() function returns a string like '0b101' for integer 5.
    # We use [2:] to remove the '0b' prefix.
    # zfill(num_bits) ensures the string has a length of num_bits, filling with zeros if needed.
    return bin(value)[2:].zfill(num_bits)

# This function converts a binary string back into its corresponding integer value.
# For example, '00101' is converted to 5.
def bin_str_to_int(bin_str):
    # The int() function can convert a string representation of a number
    # into an actual integer. The second argument, 2, tells Python that
    # the string is a binary representation.
    return int(bin_str, 2)

# This function is our objective function. It's what we're trying to optimize using the genetic algorithm.
# The function receives an individual (a possible solution to the problem) as an argument.
# This individual has a binary representation for two numbers, x and y.
def function_to_optimize(individual):
    # Extract the first 25 binary digits from the individual and convert it to a string.
    # This string represents the binary form of x.
    x_bin = ''.join(map(str, individual[:25]))

    # Extract the next 25 binary digits from the individual and convert it to a string.
    # This string represents the binary form of y.
    y_bin = ''.join(map(str, individual[25:]))

    # Convert the binary strings of x and y back to integer form.
    x = bin_str_to_int(x_bin)
    y = bin_str_to_int(y_bin)

    # Scale the integer values of x and y by dividing them by 1e6.
    # This brings the values to the desired scale for the optimization problem.
    scaled_x, scaled_y = x / 1e6, y / 1e6

    # The function we're trying to optimize. In this case, it's the sine of the
    # Euclidean distance between the points (scaled_x, scaled_y) and the origin.
    # The result is returned as a tuple, which is a common convention in DEAP for fitness values.
    return np.sin(np.sqrt(scaled_x ** 2 + scaled_y ** 2)),

# DEAP's creator module is a factory that allows for the dynamic creation of classes.
# It's a unique part of DEAP that enables us to create types that will be used in the genetic algorithms.

# Here, we're creating a custom fitness class named "FitnessMax".
# This class inherits properties from base.Fitness and is designed to maximize a single objective (weights=(1.0,)).
# The weights tuple determines whether each objective is maximized or minimized. A positive weight indicates maximization.
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Next, we create an "Individual" class that will be used to represent solutions (or chromosomes) in our genetic algorithm.
# This class inherits from Python's built-in list type and is associated with the fitness type we just created.
# It means each individual will have an associated fitness value determined by the "FitnessMax" class.
creator.create("Individual", list, fitness=creator.FitnessMax)

# The toolbox is a central feature in DEAP, acting as a container for various functions and operators used in evolutionary algorithms.
toolbox = base.Toolbox()

# Registering an attribute generator named "attr_bin" in the toolbox.
# This function will randomly return either "0" or "1" when called.
# It's used to initialize binary genes in our individuals.
toolbox.register("attr_bin", random.choice, "01")

# Registering initializers in the toolbox.
# "individual" creates an instance of our custom Individual class, with 50 genes initialized by our "attr_bin" generator.
# This means each individual will have a 50-bit binary representation.
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bin, 50)

# "population" creates a list of individuals.
# It's a convenience function to generate multiple individuals at once to form an initial population.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registering genetic operators in the toolbox.
# These operators will be used during the genetic algorithm to evaluate, select, crossover, and mutate individuals.

# "evaluate" maps to our custom objective function.
toolbox.register("evaluate", function_to_optimize)

# "mate" is set to perform two-point crossover. This crossover method selects two random points in the binary string
# and swaps the bits between these points for two individuals, creating two offspring.
toolbox.register("mate", tools.cxTwoPoint)

# "mutate" is set to perform bit-flip mutation. This mutation method flips individual bits from 0 to 1 or vice-versa
# with a certain probability (indpb=0.1, meaning each bit has a 10% chance of being flipped).
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

# "select" is set to use tournament selection. In this method, 'tournsize' individuals are randomly chosen from the population,
# and the best (with highest fitness) is selected. The process is repeated until enough individuals are chosen for the next generation.
toolbox.register("select", tools.selTournament, tournsize=4)

# Define a custom mutation function.
# This function performs a bit-flip mutation for an individual.
# Each gene (bit) in the individual has a 10% chance (indpb = 0.1) of being flipped.
def custom_mutate(individual):
    for i in range(len(individual)):
        if random.random() < 0.1:  # Mutation probability for each gene
            individual[i] = '1' if individual[i] == '0' else '0'  # Flip the bit
    # Remove the current fitness value since the individual has changed.
    del individual.fitness.values

# The following section will only be executed if the script is run directly (not imported as a module).
if __name__ == "__main__":
    # Create an initial population of 100 individuals.
    pop = toolbox.population(n=100)

    # Define the crossover probability (CXPB) and mutation probability (MUTPB).
    CXPB, MUTPB = 0.7, 0.2

    # Evaluate the fitness of the initial population.
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Begin the evolutionary loop, running for 1000 generations.
    for g in range(1000):
        # Select individuals to create the next generation.
        # The number of selected individuals equals the size of the current population.
        offspring = toolbox.select(pop, len(pop))

        # Create a deep copy of the selected individuals to ensure changes don't affect the original population.
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring.
        # Pairwise mating is performed (every two consecutive offspring are considered for crossover).
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:  # With CXPB probability, perform crossover
                toolbox.mate(child1, child2)
                # Remove the fitness values of the children since they might have changed.
                del child1.fitness.values
                del child2.fitness.values

        # Apply custom mutation on each offspring with MUTPB probability.
        for mutant in offspring:
            if random.random() < MUTPB:
                custom_mutate(mutant)

        # Re-evaluate the fitness of the offspring that were affected by crossover or mutation.
        invalids = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalids)
        for ind, fit in zip(invalids, fitnesses):
            ind.fitness.values = fit

        # The next generation becomes the offspring.
        pop[:] = offspring

    # Print the results after the evolution is complete.
    print("Evolution finished")

    # Find the best individual from the final population.
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))



/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Evolution finished
Best individual is ['1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1'], (0.9999999999999977,)




## Introduction

The idea is to use the resources and somehow techniques of the previous code  to optimize the ground state energy of a simple pairing Hamiltonian by combining the power of genetic algorithms (GAs) with quantum annealing (QA) techniques. The Hamiltonian under consideration is:
$$ H = \sum_k \epsilon_k a_k^\dagger a_k - g P^\dagger P $$
where:
* $ a_k^\dagger $ and $ a_k $ are the creation and annihilation operators, respectively.
* $ \epsilon_k $ denotes the single-particle energy.
* $ g $ is the pairing strength.
* $ P^\dagger = \sum_{k>0} a_k^\dagger a_{-k}^\dagger $ represents the pair creation operator.


## Matrix Representation

With 4 energy levels and a two-fold degeneracy per level, the matrix representation is:
\[ H = \begin{pmatrix}
2\epsilon_1 - g & -\frac{g}{2} & -\frac{g}{2} & -\frac{g}{2} \\
-\frac{g}{2} & 2\epsilon_2 - g & -\frac{g}{2} & -\frac{g}{2} \\
-\frac{g}{2} & -\frac{g}{2} & 2\epsilon_3 - g & -\frac{g}{2} \\
-\frac{g}{2} & -\frac{g}{2} & -\frac{g}{2} & 2\epsilon_4 - g \\
\end{pmatrix} \]
Eigenvalues of this matrix yield the energy spectrum.

## Genetic Algorithm and Quantum Annealing Approach

The optimization involves a GA, with the following steps:
1. **Representation**: Binary representation for each energy.
2. **Evaluation**: Evaluates the ground state energy.
3. **Selection, Crossover, and Mutation**: Uses tournament selection, two-point crossover, and bit-flip mutation.
4. **Quantum Annealing Step**: Though currently a placeholder, it suggests that quantum techniques can be integrated to potentially hasten convergence.

## Role of Quantum Annealing

Quantum Annealing is a quantum optimization technique. Unlike classical algorithms which may get stuck in local minima, QA can escape such traps by tunneling.
## Results

The algorithm iteratively refines the $$\epsilon_k $$ values to minimize the ground state energy. The final results present an optimized solution.


In [6]:
from deap import base, creator, tools
import random
import numpy as np
from scipy.linalg import eigh  # To solve for eigenvalues

# Constants
MAX_INT = 4000000
NUM_BITS = len(bin(MAX_INT)[2:])
NUM_LEVELS = 4  # Number of energy levels
g = 0.5  # Pairing strength

# Convert integer to binary array
def int_to_bin_array(value):
    return [int(x) for x in bin(value & int("1"*NUM_BITS, 2))[2:].zfill(NUM_BITS)]

# Convert binary array to integer
def bin_array_to_int(bin_array):
    return int(''.join(map(str, bin_array)), 2) - MAX_INT  # Offset by -MAX_INT to handle negatives

def pairing_hamiltonian_evaluation(individual):
    # Convert binary representation back to integer and then to a float between -4 and 4
    epsilons = [bin_array_to_int(individual[i*NUM_BITS:(i+1)*NUM_BITS])/1000000.0 for i in range(NUM_LEVELS)]

    # Construct Hamiltonian matrix
    H = np.zeros((NUM_LEVELS, NUM_LEVELS))
    for i in range(NUM_LEVELS):
        H[i, i] = 2 * epsilons[i] - g
        for j in range(i+1, NUM_LEVELS):
            H[i, j] = -g/2
            H[j, i] = -g/2

    # Compute lowest eigenvalue (ground state energy)
    eigenvalues = eigh(H, eigvals_only=True)
    ground_state_energy = eigenvalues[0]

    return -ground_state_energy,  # We want to maximize the fitness, so return negative energy
# Only create the FitnessMax and Individual classes if they don't exist
if "FitnessMax" not in creator.__dict__:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)
# Set up the genetic algorithm components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bin", random.choice, [0, 1])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bin, NUM_BITS * NUM_LEVELS)  # Individual size is now NUM_BITS * NUM_LEVELS
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", pairing_hamiltonian_evaluation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Quantum annealing step (placeholder)
def quantum_annealing_step(individuals):
    # Use a quantum annealer/simulator to evolve the individuals
    # Placeholder function for now.
    return individuals

if __name__ == "__main__":
    pop = toolbox.population(n=100)

    # Evaluate initial population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Evolutionary loop with Quantum step
    for gen in range(50):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:  # Crossover probability
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:  # Mutation probability
                toolbox.mutate(mutant)

        # Quantum annealing step
        offspring = quantum_annealing_step(offspring)

        # Evaluate individuals
        fitnesses = map(toolbox.evaluate, offspring)
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

    best_individual = tools.selBest(pop, 1)[0]
    print(f"Best solution: {[bin_array_to_int(best_individual[i*NUM_BITS:(i+1)*NUM_BITS])/1000000.0 for i in range(NUM_LEVELS)]} with fitness {best_individual.fitness.values[0]}")

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Best solution: [-3.99996, -3.999981, -3.999223, -3.999856] with fitness 9.24951038606728
